In [1]:
import kmodels as kmk
import numpy as np
import pandas as pd
import os
import torch
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
os.getcwd()

c:\Users\efons\anaconda3\envs\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'c:\\Users\\efons\\Desktop\\kalyn_lstm\\kmodels\\notebooks'

In [2]:
data_path = '../../data/'
# get all pickles
pickles = [f for f in os.listdir(data_path) if f.endswith('.pkl')]
# get the Formatted pickle
data = [f for f in pickles if 'Formatted' in f][0]
df = pd.read_pickle(data_path + data)
df

,Time,FCU,EPL,EPB,FPL,APL,FPB,OPP,Fx,Fy,Fz,ADD,Subject
0,0.000000,0.054566,0.058537,0.062743,0.155728,0.023906,0.021142,0.070465,-9.259692,-4.447818,-6.536988,0.026819,6.26_0.9_1.06
1,0.009799,0.054646,0.058400,0.062911,0.154414,0.023787,0.021018,0.077124,-9.502468,-4.600985,-6.520491,0.026349,6.26_0.9_1.06
2,0.019599,0.054800,0.057753,0.062681,0.153220,0.023763,0.021220,0.095268,-9.513043,-4.624847,-6.518658,0.026434,6.26_0.9_1.06
3,0.029398,0.056122,0.056917,0.063201,0.157919,0.023762,0.022113,0.114959,-9.409544,-4.564386,-6.497891,0.026648,6.26_0.9_1.06
4,0.039197,0.057602,0.056416,0.063981,0.163014,0.023785,0.023148,0.126942,-9.251525,-4.474644,-6.437751,0.026858,6.26_0.9_1.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345171,1.950067,0.077327,0.028705,0.070763,0.186818,0.081797,0.022118,0.090759,-4.916051,-5.716611,-12.003086,0.024750,6.53_1.04_1.02
1345172,1.959866,0.077751,0.027808,0.071065,0.191791,0.082448,0.022103,0.095652,-5.248973,-5.937422,-12.067884,0.024120,6.53_1.04_1.02
1345173,1.969666,0.078584,0.027024,0.071566,0.193814,0.083229,0.022089,0.093776,-5.556444,-6.157407,-12.143441,0.023570,6.53_1.04_1.02
1345174,1.979465,0.078977,0.026315,0.071994,0.195077,0.083762,0.022077,0.086941,-5.809779,-6.380327,-12.206728,0.023072,6.53_1.04_1.02


In [3]:
xcols = df.columns
# drop "ADD", and "Subject"
xcols = list(xcols.drop(['ADD', 'Subject']))
ycols = ['Fx','Fy','Fz']
# drop all the ycols from xcols
xcols = list(set(xcols) - set(ycols))
print(xcols)
print(ycols)
# lets normalize all the xcols data and put it back in the dataframe
scaler = StandardScaler()
x_scaled = scaler.fit_transform(df[xcols])
df[xcols] = x_scaled

['Time', 'EPB', 'APL', 'FPB', 'FCU', 'FPL', 'EPL', 'OPP']
['Fx', 'Fy', 'Fz']


In [4]:
#random_search_results = kmk.random_search(df, batch_size=10000, n_epochs=11, n_iter=10, n_splits=5)

In [5]:
#random_search_results[1]

In [6]:
model = kmk.NN(layer_size=50, layers=3, n_inputs=len(xcols), n_outputs=len(ycols))

In [7]:
data = kmk.Dataset(df, sort_column=['Subject'])


518


In [8]:
exp_df = pd.read_pickle('../../data/SeparatedData.pkl')
exp_data = kmk.Dataset(exp_df, sort_column=['Subject'])
exp_data.X_lstm.shape, exp_data.Y_lstm.shape, exp_data.Y_lstm[0].shape

8


(torch.Size([24, 204, 8]), torch.Size([24, 204, 3]), torch.Size([204, 3]))

In [9]:
# create a lstm with pytorch. the inputs should have lstom_layers=3, n_inputs=len(xcols), n_outputs=len(ycols) and n_timesteps=exp_data.X_lstm.shape[1], linear_layers=3, 
# linear_layer_size=50. create the entire model with the __init__ function, and then create the forward function.

class LSTM(nn.Module):
    def __init__(self, n_lstm_layers=3, n_lstm_outputs=50, 
                 lstm_hidden_size=3, n_inputs=8, n_outputs=3, 
                 n_timesteps=204, n_linear_layers=1, linear_layer_size=50):
        super(LSTM, self).__init__()
        self.lstm_layers = n_lstm_layers
        self.n_lstm_hidden_size = lstm_hidden_size
        self.n_lstm_outputs = n_lstm_outputs
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.n_timesteps = n_timesteps
        self.n_linear_layers = n_linear_layers
        self.linear_layer_size = linear_layer_size
        self.lstm_output_dim = n_timesteps*n_lstm_layers
        
        self.lstm = nn.LSTM(input_size=n_inputs, 
                            hidden_size=lstm_hidden_size, 
                            num_layers=n_lstm_layers)
        
        self.linear_layers = nn.ModuleList()
        self.first_linear_layer = nn.Linear(self.lstm_output_dim, linear_layer_size)
        for i in range(n_linear_layers):
            self.linear_layers.append(nn.Linear(linear_layer_size, linear_layer_size))
        self.output_layer = nn.Linear(linear_layer_size, n_outputs*self.n_timesteps)
    
    def forward(self, x):
        x, _ = self.lstm(x)
        # make sure x in flattened
        x = x.flatten()
        x = self.first_linear_layer(x)
        for layer in self.linear_layers:
            x = layer(x)
        x = self.output_layer(x)
        return x.reshape(self.n_timesteps, self.n_outputs)
    
model = LSTM()
test = torch.tensor(exp_data.X_lstm[0:1])
model(test)

C:\Users\efons\AppData\Local\Temp\ipykernel_12872\1156456482.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test = torch.tensor(exp_data.X_lstm[0:1])


tensor([[-8.0565e-02, -6.7544e-02, -3.7853e-02],
        [-7.6238e-02, -1.1457e-01, -1.4739e-01],
        [-6.8136e-02, -8.3564e-02, -1.0196e-01],
        [-2.5904e-02,  6.5443e-02, -1.5824e-01],
        [-1.5480e-01,  1.5132e-02, -8.7733e-02],
        [-8.8710e-02, -3.4445e-02,  2.3464e-02],
        [-1.0053e-01, -3.2440e-02, -6.1174e-02],
        [ 7.5752e-03, -1.2479e-01,  9.2471e-02],
        [-1.5897e-01,  1.1926e-01, -1.1268e-01],
        [-1.8198e-01,  5.3528e-02, -1.2466e-01],
        [-1.3076e-01, -9.0927e-02,  3.7879e-02],
        [ 5.7908e-02, -1.4597e-02, -1.1608e-01],
        [ 2.4974e-02, -8.4321e-02, -1.3568e-01],
        [-8.1687e-02,  6.7012e-02,  1.1693e-01],
        [ 2.6076e-01, -1.0606e-01,  1.0829e-01],
        [ 3.1080e-02, -2.8567e-01,  1.1045e-02],
        [-1.2413e-01, -1.0555e-01, -2.0011e-02],
        [ 1.4949e-01,  5.6415e-04, -1.4077e-01],
        [-4.6370e-02, -7.1377e-03,  8.7054e-02],
        [ 1.6137e-02, -1.5606e-01, -1.0999e-02],
        [-1.0742e-02

In [12]:
losses = kmk.run_Pytorch(model, data.X_lstm, data.Y_lstm, n_epochs=21, batch_size=1, learning_rate=1e-4)

EPOCH :  0 , dt:  44.19698405265808 seconds, losses : 1.7726176977157593


KeyboardInterrupt: 

In [13]:
def TLLSTM(model, X, change_layers=1):
    new = LSTM(n_lstm_layers=model.lstm_layers, n_lstm_outputs=model.n_lstm_outputs,
               lstm_hidden_size=model.n_lstm_hidden_size, n_inputs=model.n_inputs,
               n_timesteps=model.n_timesteps, n_linear_layers=model.n_linear_layers,
               linear_layer_size=model.linear_layer_size)
    test = new(X)
    new.load_state_dict(model.state_dict())
    children = [child for child in new.children()]
    for child in children:
        for param in child.parameters():
            param.requires_grad = False
    total_layers = len(children)
    for i in range(change_layers):
        layer = children[total_layers-i-1]
        layer_params = layer.parameters()
        for p in layer_params:
            p.requires_grad = True
    return new



In [14]:
tl_model = TLLSTM(model, test, change_layers=1)
tl_model

LSTM(
  (lstm): LSTM(8, 3, num_layers=3)
  (linear_layers): ModuleList(
    (0): Linear(in_features=50, out_features=50, bias=True)
  )
  (first_linear_layer): Linear(in_features=612, out_features=50, bias=True)
  (output_layer): Linear(in_features=50, out_features=612, bias=True)
)

In [ ]:
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')

In [ ]:
pred = model(data.X_lstm.to('cuda')).detach().cpu().numpy()
pred[:,0].shape, data.Y[:,0].shape

In [ ]:
x_1 = data.X[data.subject_index[0]]
plt.plot(x_1[0:204,0], x_1[0:204,1])

In [ ]:
X = data.X.float()
Y = data.Y.float()

# use a scaler to scale the data
scaler = StandardScaler()
X = scaler.fit_transform(X)
X = torch.from_numpy(X).float()
X.shape

In [ ]:
plt.plot(X[0:300,0], X[0:300,1])

In [ ]:
losses = kmk.run_Pytorch(model, X, Y, batch_size=len(X)/2, n_epochs=251, learning_rate=1e-3,
                         optimizer=torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-2))


In [ ]:
plt.plot(losses)

In [ ]:
tlm = kmk.TLNN(model, X[0:2, :], change_layers=1)


In [ ]:
df2 = pd.read_pickle('../../data/SeparatedData.pkl')
df2

In [ ]:
print([i for i in df.columns if i in xcols])
print(xcols)
exp_data = kmk.Dataset(df2, sort_column=['Subject'])
n_subjects = len(exp_data.subjects)

test_data = exp_data.X[(n_subjects-1)*900:, :]
test_labels = exp_data.Y[(n_subjects-1)*900:, :]


train_data= exp_data.X[0:(n_subjects-1)*900, :]
train_labels = exp_data.Y[0:(n_subjects-1)*900, :]
train_data = train_data.float()
train_labels = train_labels.float()
# scale the data
train_data = train_data.detach().cpu().numpy()
train_data = scaler.transform(train_data)
train_data = torch.from_numpy(train_data).float()

In [ ]:
exp_data.X.shape

In [ ]:
tlm_losses = kmk.run_Pytorch(tlm, train_data, train_labels, learning_rate=1e-3, 
                             batch_size=32, n_epochs=1001, 
                             optimizer=torch.optim.Adam(tlm.parameters(), lr=1e-3, weight_decay=1e-1))


In [ ]:
# parity plot
pred = tlm(test_data.float().to('cuda'))
plt.scatter(pred.detach().cpu().numpy()[:,0], test_labels.detach().cpu().numpy()[:,0])
r2 = kmk.r2_score(pred.detach().cpu().numpy()[:,0], test_labels.detach().cpu().numpy()[:,0])
mae = kmk.mean_absolute_error(pred.detach().cpu().numpy()[:,0], test_labels.detach().cpu().numpy()[:,0])
mse = kmk.mean_squared_error(pred.detach().cpu().numpy()[:,0], test_labels.detach().cpu().numpy()[:,0])
print('r2: ', r2)
print('mae: ', mae)
print('mse: ', mse)

In [ ]:
# random forest for comparison
from sklearn.ensemble import RandomForestRegressor
reg = RandomForestRegressor(n_estimators=1000, max_depth=10, random_state=0, n_jobs=-1)
reg.fit(train_data.detach().cpu().numpy(), train_labels.detach().cpu().numpy())
pred = reg.predict(test_data.detach().cpu().numpy())
# parity plot
plt.scatter(pred[:,0], test_labels.detach().cpu().numpy()[:,0])
# print out all the stats
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
r2 = r2_score(test_labels.detach().cpu().numpy(), pred)
mae = mean_absolute_error(test_labels.detach().cpu().numpy(), pred)
mse = mean_squared_error(test_labels.detach().cpu().numpy(), pred)
print('R2: ', r2)
print('MAE', mae)
print('MSE', mse)

In [ ]:
# lets try a linear model
# grab the x data for the first subject
x, y = kmk.Dataset(df2, sort_column=['Subject'])[0]
# sort x and y by the time column
time = x[:,0]
x = x[time.argsort()]
y = y[time.argsort()]
plt.plot(x[:,0], y[:,0])
plt.plot(x[:,0], y[:,1])
plt.plot(x[:,0], y[:,2])

# lets plot the predictions from the tlm model
x_1 = scaler.transform(x)
pred_1 = tlm(torch.from_numpy(x_1).float().to('cuda')).detach().cpu().numpy()
x_1 = scaler.inverse_transform(x_1)
plt.plot(x[:,0], pred_1[:,0],alpha=0.5)
plt.plot(x[:,0], pred_1[:,1],alpha=0.5)
plt.plot(x[:,0], pred_1[:,2],alpha=0.5)